[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aarondomenzain/tracking-softmatter-aarond/blob/tracking-softmatter-aarond-dev/tutorial/tracking/tracking_spheres.ipynb)

# Particle Tracking Tutorial: Trajectory Linking
In this tutorial, you’ll explore different methods to link particle localizations across time to recostruct trajectories — using both simulated data and real experimental images.

You’ll start by generating a simulated movie of microscopic particles undergoing Brownian motion, mimicking what you might see in a soft matter or biophysics experiment. For each frame, you'll use LodeSTAR, a self-supervised neural network, to detect and localize particles. Then comes the core challenge: linking localization into trajectories.

Here’s what you’ll test and compare:

- Nearest-neighbor linking (using TrackPy — a classic in particle tracking)

- Linear Assignment Problem (LAP) (using LapTrack - a more flexible and general framework)

- MAGIK (a geometric deep learning method based on graph neural networks)

You’ll be using Python libraries like NumPy, SciPy, Matplotlib, scikit-image, PyTorch, DeepTrack, and Deeplay. 

## Table of Contents

0. [Importing the Required Libraries and Loading Utility Functions](#importing-the-required-libraries-and-loading-utility-functions)
1. [Loading and Visualizing Experimental Videos](#loading-and-visualizing-experimental-videos)
2. [Simulating Realistic Videos with DeepTrack](#simulating-realistic-videos-with-deeptrack)
    - [Simulating a Single Particle](#simulating-a-single-particle)
    - [Simulating a Video Frame](#simulating-a-video-frame)
    - [Simulating Brownian Trajectories](#simulating-brownian-trajectories)
    - [Simulating a Video](#simulating-a-video)

2. [Detecting and Localizing Particles with LodeSTAR](#detecting-and-localizing-particles-with-lodestar)
    - [Training LodeSTAR with Experiments](#training-lodestar-with-experiments)
    - [Evaluating LodeSTAR on Simulations](#evaluating-lodestar-on-simulations)
    - [Applying LodeSTAR to Simulations](#applying-lodestar-to-simulations)
    - [Applying LodeSTAR to Experiments](#applying-lodestar-to-experiments)

3. [Method 1: Nearest-neighbor Linking with TrackPy](#method-1-nearest-neighbor-linking-with-trackpy)
    - [Linking Localizations in Simulations](#linking-localizations-in-simulations)
    - [Evaluating Linking Performance](#evaluating-linking-performance)
    - [Linking Localizations in Experiments](#linking-localizations-in-experiments)

4. [Method 2: Linear Assignment Problem (LAP) with LapTrack](#method-2-linear-assignment-problem-lap-with-laptrack)
    - [Linking Localizations in Simulations](#linking-localizations-in-simulations)
    - [Evaluating Linking Performance](#evaluating-linking-performance)
    - [Linking Localizations in Experiments](#linking-localizations-in-experiments)

5. [Method 3: MAGIK](#method-3-magik)
    - [Training MAGIK with Simulations](#training-magik-with-simulations)
    - [Linking Localizations in Simulations](#linking-localizations-in-simulations)
    - [Evaluating Linking Performance](#evaluating-linking-performance)
    - [Linking Localizations in Experiments](#linking-localizations-in-experiments)



## Importing the Required Libraries and Loading Utility Functions

Uncomment the next cell if running on Google Colab/Kaggle.

In [ ]:
#!pip install deeptrack deeplay trackpy laptrack -q

In [ ]:
# Standard libraries.
import os
import random
import sys
import logging

# Configuration
import matplotlib
matplotlib.rcParams["animation.embed_limit"] = 60 # Allow larger animations inline
logging.disable(logging.WARNING) # Suppress warnings and below

# Core Scientific Stack
import numpy as np
import pandas as pd

# Plotting and Display
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Machine Learning
import torch
from torchvision.transforms import Compose
from torch_geometric.loader import DataLoader
from sklearn.metrics import f1_score
import deeplay as dl

# Particle Tracking and Simulation
import deeptrack as dt
import trackpy as tp
from laptrack import LapTrack

Load a set of custom functions defined specifically for this notebook from the `utils` directory. For detailed documentation of each function, refer to the comments and docstrings within the files.

In [ ]:
# Load functions and utilities for dataset generation and visualization.
# Sys append a folder to the path.
sys.path.append(os.path.abspath(os.path.join("..", "..")))

# Import all the functions contained in the folder utils.
import utils

Set random seeds to make results reproducible across runs, especially during training and data simulation.

In [ ]:
# Set a fixed seed value
seed = 98

# Python, NumPy, and PyTorch (CPU)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Only set CUDA seeds if a GPU is available
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print(f"Seeds set to {seed} (with CUDA: {torch.cuda.is_available()})")

## Loading and Visualizing Experimental Videos

You'll use experimental video of a system of colloidal particles recorded with fluorescence microscopy. Data from https://www.nature.com/articles/s41467-022-30497-z.


In [ ]:
# Define the folder and video file name.
video_folder = "videos"
video_file_name = "experimental_video.npy"

# Construct the full path.
video_path = os.path.join(video_folder, video_file_name)

# Load the video data.
exp_video = np.load(video_path)

utils.play_video(exp_video, "Experimental Video")

Display the first frame of the video, manually select and display a single particle by specifying its centroid coordinates (x, y) and a box width.

In [ ]:
# Select the first frame of the video.
exp_image = exp_video[0]

# Get the shape of the image.
assert exp_image.shape[0] == exp_image.shape[1], "Warning: Image is not square!"
exp_image_size = exp_image.shape[0]

Specify the parameters for the viewing box.

In [ ]:
# Box size to zoom in an individual particle.
exp_crop_size = 15

# Coordinates of the center of the particle.
x_center = 96
y_center = 41

In [ ]:
# Calculate top-left corner of the crop.
x = x_center - exp_crop_size // 2
y = y_center - exp_crop_size // 2

# Select a crop as a subset of the entire image.
exp_crop = exp_image[y:y + exp_crop_size, x:x + exp_crop_size]  # row (y), column (x)

# Initialize figure instance.
fig = plt.figure()

vmin, vmax = np.percentile(exp_image, [1, 99])

# Draw a red rectangle around the crop.
fig.add_subplot(111)
plt.imshow(exp_image, cmap='gray', vmin=vmin, vmax=vmax)
plt.title("Experimental Image", size=13)
plt.plot([x, x, x+exp_crop_size, x+exp_crop_size, x],[y, y+exp_crop_size, y+exp_crop_size, y, y], 'r-')
plt.axis('off')

# Plot the rectangle on the top right corner.
fig.add_subplot(555)
plt.imshow(exp_crop, cmap='gray', vmin=vmin, vmax=vmax)
plt.axis('off')
plt.show()

## Simulating Realistic Videos with DeepTrack

DeepTrack allows you to simulate physically realistic microscopy images, enabling precise control over imaging parameters and particle properties. These simulations provide ground-truth data, making them ideal for benchmarking classical and AI-based tracking methods, as well as for training neural networks in a controlled environment.

### Simulating a Single Particle

Adjust the simulation parameters to accurately replicate the features observed in the cropped region of the experimental image.

In [ ]:
# Same as the box width.
sim_crop_size = exp_crop_size  

# Size of a pixel in nanometers in the output image.
pixel_size_nm = 100 # In nm.

# Radius of the particle.
particle_radius = 200 # In nm.

# Define central spherical scatterer.
sphere = dt.Sphere(
    position=0.5 * np.array([sim_crop_size, sim_crop_size]) + (-0.5, 0.5),
    z= 0 * dt.units.nm, # Particle in focus.
    radius= particle_radius * dt.units.nm,  # Radius in nanometers.
    intensity= 0.9E5,  # Field magnitude squared.
)

# Simulate the properties of the fluorescence microscope.
optics = dt.Fluorescence(
    NA=1.0,  # Numerical aperture.
    wavelength=638 * dt.units.nm,
    refractive_index_medium=1.33,
    output_region=[0, 0, sim_crop_size, sim_crop_size],
    magnification=1,
    resolution=pixel_size_nm * dt.units.nm, # Camera resolution or effective resolution.
)

# Apply transformations.
sim_crop = (
    optics(sphere)
    >> dt.Background(750)
    >> dt.Poisson(snr=6500)
)

# Turn the crop into a NumPy array.
sim_crop = np.squeeze(sim_crop())

# Plot the simulated and experimental crops.
fig, axes = plt.subplots(1, 2)

# Simulated crop.
plot = axes[0].imshow(sim_crop, cmap="gray")
axes[0].axis("off")
axes[0].set_title("Simulated Crop")

# Experimental crop.
axes[1].imshow(exp_crop, cmap="gray")  
axes[1].axis("off")
axes[1].set_title("Experimental Crop")

# Adjust layout and show plot.
plt.tight_layout()
plt.show()

Extract and visualize the raw intensity profiles along the central horizontal and vertical lines of both the simulated and experimental crops. This comparison helps evaluate how well the simulation reproduces the intensity distribution observed in real microscopy images.

In [ ]:
# Compute the center index.
center = sim_crop_size // 2

# Extract horizontal (row) profiles.
sim_horiz = sim_crop[center, :]
exp_horiz = exp_crop[center, :]

# Extract vertical (column) profiles.
sim_vert = sim_crop[:, center]
exp_vert = exp_crop[:, center]

# Create a 1×2 subplot.
fig, axes = plt.subplots(1, 2, figsize=(12, 4), tight_layout=True)

# --- Horizontal profile ---
axes[0].plot(sim_horiz, label="Simulated Crop", color="orange")
axes[0].plot(exp_horiz, label="Experimental Crop", color="blue")
axes[0].set_xlabel("Pixel (x)")
axes[0].set_ylabel("Intensity")
axes[0].set_title("Horizontal Intensity Profile (Center Row)")
axes[0].legend()
axes[0].grid(True, linestyle="--", alpha=0.5)

# --- Vertical profile ---
axes[1].plot(sim_vert, label="Simulated Crop", color="orange")
axes[1].plot(exp_vert, label="Experimental Crop", color="blue")
axes[1].set_xlabel("Pixel (y)")
axes[1].set_ylabel("Intensity")
axes[1].set_title("Vertical Intensity Profile (Center Column)")
axes[1].legend()
axes[1].grid(True, linestyle="--", alpha=0.5)


### Simulating a Video Frame

Create a simulated image containing non-overlapping spherical particles. Begin by generating their coordinates—these will serve as the ground-truth positions. Then, use DeepTrack to render optically realistic particles at these coordinates, resulting in a physically plausible microscopy image.

In [ ]:
# Parameters of the simulation.
sim_image_size = exp_image_size
N_particles = 60
particle_radius = 200  # Particle radius in nm.

# Dictionary for particle properties. Dimensions are set with lambda
# functions to introduce variety to the dataset.
sphere_properties = {
    "intensity": lambda: np.random.normal(1.1, 0.3)*0.9E5,
    "z": lambda: np.random.uniform(0, 3000) * dt.units.nm,
    "radius": particle_radius * dt.units.nm,
}

# Set the optical properties of the microscope.  
optics_properties = dt.Fluorescence(
    NA=1.0,  # Numerical aperture.
    wavelength=638 * dt.units.nm,
    refractive_index_medium=1.33,
    output_region=[0, 0, sim_image_size, sim_image_size],
    magnification=1,
    resolution=pixel_size_nm * dt.units.nm, # Camera resolution or effective resolution.
)

# Generate ground truth positions.
sim_gt_pos = utils.generate_centroids(
    num_particles=N_particles,
    fov_size=sim_image_size,
    particle_radius=particle_radius,
)

# Simulate image.
sim_image = utils.transform_to_video(
    sim_gt_pos,
    fov_size=sim_image_size,
    core_particle_props=sphere_properties,
    optics_props=optics_properties,
    background_props={"poisson_snr": 6500, "background_mean": 750},
)

Visualize the simulated image and compare it with the experimental one.

In [ ]:
# Plot the simulated and experimental images.
fig, axes = plt.subplots(1, 2)

vmin, vmax = np.percentile(exp_image, [1, 99])

# Simulated image.
plot = axes[0].imshow(sim_image, cmap="gray", vmin=vmin, vmax=vmax)
axes[0].axis("off")
axes[0].set_title("Simulated Image")

# Experimental image.
axes[1].imshow(exp_image, cmap="gray", vmin=vmin, vmax=vmax)  
axes[1].axis("off")
axes[1].set_title("Experimental Image")

# Adjust layout and show plot.
plt.tight_layout()
plt.show()

Perform a more quantitative comparison by plotting the intensity histograms.

In [ ]:
# Flatten the arrays to 1D.
sim_vals = sim_image.ravel()
exp_vals = exp_image.ravel()

# Compute common bin edges.
all_vals = np.concatenate([sim_vals, exp_vals])
num_bins = 60
bins = np.linspace(all_vals.min(), np.quantile(all_vals, 0.99), num_bins + 1)

# Create figure with two subplots sharing axes.
fig, axes = plt.subplots(
    1, 2,
    figsize=(10, 4),
    sharey=True, sharex=True,
    tight_layout=True
)

# Histogram for simulated image.
axes[0].hist(
    sim_vals,
    bins=bins,
    alpha=0.7,
    edgecolor="black"
)
axes[0].set_title("Simulated Image Histogram")
axes[0].set_xlabel("Intensity")
axes[0].set_ylabel("Pixel Count")

# Histogram for experimental image.
axes[1].hist(
    exp_vals,
    bins=bins,
    alpha=0.7,
    edgecolor="black"
)
axes[1].set_title("Experimental Image Histogram")
axes[1].set_xlabel("Intensity")

plt.show()

### Simulating Brownian Trajectories

You'll simulate a set of trajectories that visually resemble the experimental data to evaluate the performance of different tracking methods. The goal is to replicate the Brownian motion of nanoparticles as observed in the experimental videos. This is done using the `simulate_Brownian_trajs` function from the utility file, which generates 2D trajectories based on a random walk model. Refer to the function for more details.

In [ ]:
# Simulation parameters.
number_particles = 60
number_timesteps = 50

# Simulate trajectories for one video.
sim_trajs_gt = utils.simulate_Brownian_trajs(
    num_particles = number_particles,
    num_timesteps = number_timesteps,
    fov_size = sim_image_size,
    diffusion_std=0.5, # Corresponds to sqrt(2Dt),
)

For evaluation purposes, trajectories that move out and back in the field of view due to boundary conditions are treated as separate trajectories. For further analysis, the trajectories are transformed into a list.

In [ ]:
# Break trajectories going in/out of FOV.
sim_trajs_gt_list = utils.traj_break(
    trajs = sim_trajs_gt,
    fov_size = sim_image_size,
    num_particles = sim_trajs_gt.shape[1],
)

### Simulating a Video

You'll simulate a video of particle motion that resembles experimental data and compares the two by playing them simultaneously.

**Note:** Several out-of-focus particles are rather dim and not clearly visible unless the dynamic range of the video is adjusted. However, including them is essential to accurately reproduce the experimental conditions, especially in terms of intensity distribution and background noise characteristics.

In [ ]:
# Simulate video.
sim_video = utils.transform_to_video(
    np.delete(sim_trajs_gt, 2, 2), # Remove frame axis.
    fov_size=sim_image_size,
    core_particle_props=sphere_properties,
    optics_props=optics_properties,
    background_props={"poisson_snr": 6500, "background_mean": 750},
    save_video=True,
    path="videos/simulated_video.tiff",
)

# Play both videos and compare.
utils.play_video(sim_video, "Simulated Video")
utils.play_video(exp_video, "Experimental Video")

## Detecting and Localizing Particle with LodeSTAR

LodeSTAR will be used to detect the particles' positions at each frame of the video, similarly as shown in the **Detections** notebooks of this tutorial. These positions will be passed to different linking methods to build trajectories and compare their performance.

### Training LodeSTAR with Experiments

For the training, you'll start by obtaining crops of our particles.

In [ ]:
# Define crop parameters: (x, y, width)
crop_params = [
    (196, 218, 20),
    (97, 144, 20),
    (232, 233, 20),
    (59, 203, 20),
    (202, 70, 20),
]

# Extract crops from the image.
exp_crops = [exp_image[y:y + w, x:x + w] for x, y, w in crop_params]

# Plot up to 18 crops.
utils.plot_crops(np.squeeze(exp_crops), title="Experimental Crops")


Prepare the training pipeline with the selected experimental crops. If your crop is large (i.e larger than64x64), you can make the training process faster by using meanpooling in the training pipeline with `>> dt.Pool(np.mean, 2)`, which is added but commented out in the pipeline below.

In [ ]:
# Random selector of crops.
random_crop = lambda: random.choice(np.array(exp_crops))

# Define training pipeline with augmentations.
train_pipeline = (
    dt.Value(random_crop)
    >> dt.Affine(scale=lambda: np.random.uniform(0.8, 1.2))
    # >> dt.Pool(np.mean, 2)
    >> dt.NormalizeMinMax(0.0, 1.0)
    >> dt.Gaussian(0, 0.05)
    >> dt.NormalizeMinMax(
        lambda: np.random.uniform(0.0, 0.1), 
        lambda: np.random.uniform(0.9, 1.0),
        )
    >> dt.MoveAxis(-1, 0)
    >> dt.pytorch.ToTensor(dtype=torch.float32)
)

# Define dataset.
train_dataset = dt.pytorch.Dataset(
    train_pipeline,
    length=512,
)

# Define Dataloader.
dataloader = dl.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
)

Create a LodeSTAR model with 4 random geometric transformations per input and set up the trainer, which will handle the training loop. It runs for 50 epochs.

In [ ]:
# Initiate LodeSTAR.
lodestar = dl.LodeSTAR(
    n_transforms=4,
    optimizer=dl.Adam(lr=1e-3),
).build()

# Set up the trainer and specify number of epochs.
trainer_lodestar = dl.Trainer(max_epochs=50, accelerator="auto")

Check whether pre-trained LodeSTAR weights already exist on disk. If they are missing or if training is explicitly forced, the model is trained on the experimental crops using the specified training pipeline, and the resulting weights are saved for future use.

If the weights are already available and training is not forced, they are simply loaded from file.

Afterward, the model is set to evaluation mode, which disables training-specific behaviors, ensuring consistent behavior during inference.

In [ ]:
# Define whether to force training even if weights exist.
force_training = True

# Define folder and path for storing LodeSTAR experimental weights.
folder_name = "LodeSTAR"
lodestar_path = os.path.join(folder_name, "lodestar_weights")

# Train or load weights.
if not os.path.isfile(lodestar_path) or force_training:
    print("Training LodeSTAR on experimental data (either forced or weights not found).")
    
    # Ensure the save directory exists.
    os.makedirs(folder_name, exist_ok=True)

    # Train the model.
    trainer_lodestar.fit(lodestar, dataloader)

    # Save weights.
    torch.save(lodestar.state_dict(), lodestar_path)
    print(f"Saved experimental LodeSTAR weights to '{lodestar_path}'.")
else:
    # Load pre-trained weights.
    lodestar.load_state_dict(torch.load(lodestar_path, weights_only=True))
    print(f"Loaded preexisting LodeSTAR weights from '{lodestar_path}'.")

# Set model to evaluation mode.
lodestar.eval();

### Evaluating LodeSTAR on Simulations

Apply the trained LodeSTAR model to a frame of the simulated video.  Set the inference parameters which control detection sensitivity. Get the prediction features, which can be useful for visualizing detections. Extract the final coordinates of the detected particles and print how many particles were detected. Plot the predicted localizations from LodeSTAR alongside the ground truth on the simulated image and quantify the performance.

**Note:** Although the performance metrics might appear low, this is expected. LodeSTAR accurately detects all particles that are in focus and visually similar to the ones used for training. However, the simulated video includes many out-of-focus particles, which are difficult to observe unless the image’s dynamic range is carefully adjusted. These blurred particles are not detected because their appearance differs significantly from the in-focus training crops, and they were not included in LodeSTAR’s training set.

In [ ]:
# select frame number
frame_index = 0

# Format according to the (N,C,X,Y) convention.
sim_frame_formatted = utils.format_image(sim_video[frame_index])

# Parameters for inference.
alpha = 0.01
beta = 1 - alpha
cutoff = 0.01

# Get localizations from experimental image.
sim_locs_pred = lodestar.detect(
    sim_frame_formatted,
    alpha=alpha,
    beta=beta,
    mode="constant",
    cutoff=cutoff,
)[0]

# Print detection number for reference.
print(f"Found {len(sim_locs_pred[:,1])} detections.")

utils.plot_predicted_positions(
   image=sim_frame_formatted[0, 0, :, :],
   pred_positions= sim_locs_pred,
   gt_positions=sim_trajs_gt[frame_index],
   title="Lodestar Trained on Experimental Crops - Simulated Image",
)

#  Evaluate performance.
utils.evaluate_locs(sim_locs_pred, sim_trajs_gt[frame_index], distance_th=particle_radius);

### Applying LodeSTAR to Simulations

Iteratively apply LodeSTAR to every frame of the simulated video and store localizations in a dataframe.

In [ ]:
# Define a dataframe to store all the LodeSTAR detections.
df_sim_video = []
for frame_index, sim_frame in enumerate(sim_video):

    # Format image for LodeSTAR: shape must be (N, C, X, Y).
    sim_frame_formatted = utils.format_image(sim_frame)

    # Get detections from LodeSTAR.
    sim_locs_pred = lodestar.detect(
        sim_frame_formatted,
        alpha=alpha,
        beta=beta,
        cutoff=cutoff,
        mode="constant",
    )[0]

    # Store detections in a DataFrame.
    df_frame = pd.DataFrame(sim_locs_pred, columns=["x", "y"])
    df_frame["frame"] = frame_index
    df_sim_video.append(df_frame)

    # Print no. of detections every 10 frames.
    if frame_index % 10 == 0:
        print(f"Detections in frame {frame_index}: {len(sim_locs_pred)}")

# Combine all detections into a single DataFrame.
df_sim_video = pd.concat(df_sim_video, ignore_index=True)

### Applying LodeSTAR to Experiments

Iteratively apply LodeSTAR to every frame of the experimental video and store localizations in a dataframe.

In [ ]:
# Define a dataframe to store all the LodeSTAR detections.
df_exp_video = []
for frame_index, exp_frame in enumerate(exp_video):

    # Format image for LodeSTAR: shape must be (N, C, X, Y).
    exp_frame_formatted = utils.format_image(exp_frame)

    # Get detections from LodeSTAR.
    exp_locs_pred = lodestar.detect(
        exp_frame_formatted,
        alpha=alpha,
        beta=beta,
        cutoff=cutoff,
        mode="constant",
    )[0]

    # Store detections in a DataFrame.
    df_frame = pd.DataFrame(exp_locs_pred, columns=["x", "y"])
    df_frame["frame"] = frame_index
    df_exp_video.append(df_frame)

    # Print no. of detections every 10 frames.
    if frame_index % 10 == 0:
        print(f"Detections in frame {frame_index}: {len(exp_locs_pred)}")

# Combine all detections into a single DataFrame.
df_exp_video = pd.concat(df_exp_video, ignore_index=True)

## Method 1: Nearest-Neighbor Linking with TrackPy

TrackPy constructs trajectories by linking localized positions across frames using a predictive nearest-neighbor algorithm. The input must be a pandas DataFrame containing the particle positions, usually with columns `x`, `y`, and `frame`.

See the [TrackPy tutorial on prediction and linking](https://soft-matter.github.io/trackpy/dev/tutorial/prediction.html) for more details on how the algorithm works and how to tune parameters like `search_range` and `memory`.

### Linking Localizations in Simulations

Apply the method to the localization dataframe.

In [ ]:
# Link detections across frames into trajectories using trackpy.link().
# The `search_range` parameter sets the maximum allowed displacement (in pixels)
# between frames, and `memory` allows particles to vanish for a given number
# of frames and still be linked to the same trajectory.
sim_trajs_pred_method1 = tp.link(
    df_sim_video,
    search_range=20,
    memory=3,
)

Create a trajectory list from the output dataframe.

In [ ]:
sim_trajs_pred_method1_list = []
for i in sim_trajs_pred_method1.particle.drop_duplicates():
    traj = sim_trajs_pred_method1.loc[sim_trajs_pred_method1.particle == i,
     ["frame", "x", "y"]].values
    sim_trajs_pred_method1_list.append(traj)

Create a video with overlayed localizations and trajectories.

In [ ]:
sim_video_method1_results = utils.make_video_with_trajs(
    trajs_pred_list = sim_trajs_pred_method1_list,
    video = sim_video,
    fov_size = sim_image_size,
    trajs_gt_list = sim_trajs_gt_list,
)

# Display the video.
sim_video_method1_results

### Evaluating Linking Performance

Evaluate the overall performance of the tracking (detection + linking) method using the following metrics:

- **TP (True Positives):** Number of ground-truth particles correctly matched to estimated positions.

- **FP (False Positives):** Number of estimated particles that do not correspond to any ground-truth particle.

- **FN (False Negatives):** Number of ground-truth particles that were not matched to any estimated position.

- **α:** A measure of the overall agreement between ground-truth and estimated tracks, ignoring unmatched (spurious) estimated tracks.

- **β:** A stricter version of α that penalizes unmatched (spurious) tracks, providing a more realistic performance score.  

See the detailed definitions in [Chenouard et al., Nature Methods, 2014](https://www.nature.com/articles/nmeth.2808).


In [ ]:
# Evaluate performance metrics.
utils.trajectory_metrics(
    sim_trajs_gt_list,
    sim_trajs_pred_method1_list,
    eps=5,
);

Display the reconstructed trajectories together with the groud truth.

In [ ]:
#  Compute the total squared distance between all trajectories to match
#  predicted trajectories with ground truth.
matched_pairs, _, _ = utils.trajectory_assignment(
    sim_trajs_gt_list,
    sim_trajs_pred_method1_list,
    eps=5,
)

# Plot the trajectories.
utils.plot_trajectory_matches(sim_trajs_gt_list, sim_trajs_pred_method1_list, matched_pairs)

Calculate the time-averaged MSD for all the trajectories and compare curves obtained for matching trajectories (same color).

In [ ]:
utils.plot_TAMSDs(
    trajs_pred = sim_trajs_pred_method1_list,
    trajs_gt = sim_trajs_gt_list,
    matched_pairs = matched_pairs,
) 

### Linking Localizations in Experiments

Apply the same steps to track the experiment and visualize the results.

In [ ]:
# Link detections across frames into trajectories using trackpy.link().
exp_trajs_pred_method1 = tp.link(
    df_exp_video,
    search_range=20, 
    memory=3,
)

# Create a list to store trajectories.
exp_trajs_pred_method1_list = []
for i in exp_trajs_pred_method1.particle.drop_duplicates():
    traj = exp_trajs_pred_method1.loc[exp_trajs_pred_method1.particle == i,
     ["frame", "x", "y"]].values
    exp_trajs_pred_method1_list.append(traj)


exp_video_method1_results = utils.make_video_with_trajs(
    trajs_pred_list = exp_trajs_pred_method1_list,
    video = exp_video,
    fov_size = exp_image_size,
)

# Display the video.
exp_video_method1_results

Calculate the time-averaged MSD for the trajectories.

In [ ]:
utils.plot_TAMSDs(
    trajs_pred = exp_trajs_pred_method1_list,
) 

## Method 2: Linear Assignment Problem (LAP) with LapTrack


LapTrack solves the trajectory linking problem by formulating it as a Linear Assignment Problem (LAP), a well-established optimization approach in multi-object tracking. It builds a cost matrix that quantifies dissimilarity—typically based on spatial distance—between particle detections in consecutive frames. Lower distances correspond to lower costs and indicate higher likelihoods of correspondence.

LapTrack uses the Hungarian algorithm to solve this assignment problem efficiently, minimizing the total cost across the matrix. This allows it to determine the globally optimal set of assignments across frames, enabling robust trajectory reconstruction even under challenging conditions such as high particle density or noisy detections.

Examples and tutorials using LapTrack are available in the [LapTrack documentation](https://github.com/yfukai/laptrack/tree/main/docs/examples).

### Linking Localizations in Simulations

Apply the method to the localization dataframe.

In [ ]:
# Set a maximum distance in pixel units
laptrack = LapTrack(
    track_cost_cutoff=20,
    gap_closing_max_frame_count=2,
)

sim_trajs_pred_method2, _, _ = laptrack.predict_dataframe(
    df_sim_video,
    ["x", "y"],
    only_coordinate_cols=True,
)

sim_trajs_pred_method2 = sim_trajs_pred_method2.reset_index()


Create a trajectory list from the output dataframe.

In [ ]:
sim_trajs_pred_method2_list=[]
for i in sim_trajs_pred_method2.track_id.drop_duplicates():
    traj = sim_trajs_pred_method2.loc[sim_trajs_pred_method2.track_id == i,
     ["frame", "x", "y"] ].values
    sim_trajs_pred_method2_list.append(traj)

Create a video with overlayed localizations and trajectories.

In [ ]:
sim_video_method2_results = utils.make_video_with_trajs(
    trajs_pred_list = sim_trajs_pred_method2_list,
    video = sim_video,
    fov_size = sim_image_size,
    trajs_gt_list = sim_trajs_gt_list,
)

# Display the video.
sim_video_method2_results

### Evaluating Linking Performance

Evaluate the overall performance of the tracking.

In [ ]:
# Evaluate performance metrics.
utils.trajectory_metrics(
    sim_trajs_gt_list,
    sim_trajs_pred_method2_list,
    eps=5,
);

Display the reconstructed trajectories together with the ground truth.

In [ ]:
#  Compute the total squared distance between all trajectories to match
#  predicted trajectories with ground truth.
matched_pairs, _, _ = utils.trajectory_assignment(
    sim_trajs_gt_list,
    sim_trajs_pred_method2_list,
    eps=5,
)

# Plot the trajectories.
utils.plot_trajectory_matches(sim_trajs_gt_list, sim_trajs_pred_method2_list, matched_pairs)

Calculate the time-averaged MSD for all the trajectories and compare curves obtained for matching trajectories (same color).

In [ ]:
utils.plot_TAMSDs(
    trajs_pred = sim_trajs_pred_method2_list,
    trajs_gt = sim_trajs_gt_list,
    matched_pairs = matched_pairs,
) 

### Linking Localizations in Experiments

Apply the same steps to track the experiment and visualize the results.

In [ ]:
# Link detections across frames into trajectories using laptrack.
exp_trajs_pred_method2, _, _ = laptrack.predict_dataframe(
    df_exp_video,
    ["x", "y"],
    only_coordinate_cols=True,
)

exp_trajs_pred_method2 = exp_trajs_pred_method2.reset_index()


# Create a list to store trajectories.
exp_trajs_pred_method2_list=[]
for i in exp_trajs_pred_method2.track_id.drop_duplicates():
    traj = exp_trajs_pred_method2.loc[exp_trajs_pred_method2.track_id == i,
     ["frame", "x", "y"] ].values
    exp_trajs_pred_method2_list.append(traj)


exp_video_method2_results = utils.make_video_with_trajs(
    trajs_pred_list = exp_trajs_pred_method2_list,
    video = exp_video,
    fov_size = exp_image_size,
)

# Display the video.
exp_video_method2_results

Calculate the time-averaged MSD for the trajectories.

In [ ]:
utils.plot_TAMSDs(
    trajs_pred = exp_trajs_pred_method2_list,
) 

## Method 3: MAGIK

MAGIK is a tracking framework designed to analyze the motion of dynamic systems, including cells, bacteria, individual molecules, colloids, and other active particles. The name stands for Motion Analysis through Graph Neural Network Inductive Knowledge.

At its core, MAGIK uses graph neural networks (GNNs) to learn patterns in particle movement and to infer trajectories across frames. This data-driven approach enables MAGIK to outperform traditional tracking methods in challenging conditions, such as dense particle fields, complex interaction dynamics, or non-Brownian motion.

Thanks to its ability to learn and generalize motion priors, MAGIK is particularly effective in noisy or ambiguous experimental settings, making it a strong complement—or even an alternative—to classical tools like TrackPy and LapTrack.

For more details, see the publication:  
[Geometric Deep Learning Reveals the Spatiotemporal Features of Microscopic Motion](https://www.nature.com/articles/s42256-022-00595-0) – *Nat Mach Intell* **5**, 71–82 (2023).

### Training MAGIK with Simulations

To train MAGIK effectively, you first need to generate appropriate training data. Since the experimental dataset in this case features colloids undergoing Brownian motion, you'll use the `simulate_Brownian_trajs` function to produce groups of synthetic trajectories that replicate this behavior.

**Note:** It is crucial that the simulated training data accurately reflect the motion characteristics of your experimental particles. If your system exhibits pure diffusion (Brownian motion), the training simulations should mirror that. Conversely, if your experimental data involve additional dynamics—such as drift, confinement, or driven flow (e.g. in nanofluidic systems)—these should be incorporated into the training data to ensure MAGIK learns the correct motion priors.

In [ ]:
# Parameters for training dataset.
train_dataset_size = 20 # Number of videos.
train_number_particles = 30 # Number of particles per video.
train_number_timesteps = 50 # Number of frames per video.
train_fov_size = 64 # Size of the field of view.

# Initiate a dataframe containing all the simulated trajectories.
df_train_dataset = []
for video_index in range(train_dataset_size):
    # Simulate trajectories for one video.
    sim_trajs_train_dataset = utils.simulate_Brownian_trajs(
        num_particles=train_number_particles,
        num_timesteps=train_number_timesteps,
        fov_size=train_fov_size,
        diffusion_std=0.5,
    )
    # Break trajectories going in/out of FOV.
    sim_trajs_train_dataset_list = utils.traj_break(
        trajs=sim_trajs_train_dataset,
        fov_size=train_fov_size,
        num_particles=train_number_particles,
    )
    # Make into dataframe with "frame" (which frame in the video),
    #  label(which particle in that frame), set (which video).
    for traj_index, traj in enumerate(sim_trajs_train_dataset_list):
        df_traj = pd.DataFrame(
            traj[:, 1:],
            columns=["centroid-0", "centroid-1"],
        )
        df_traj["frame"] = traj[:, 0].astype(int)
        df_traj["label"] = traj_index
        df_traj["set"] = f"{video_index}"
        df_train_dataset.append(df_traj)

# Concatenate to dataframe.
df_train_dataset = pd.concat(df_train_dataset, ignore_index=True)


Normalize the trajectory coordinates between 0 and 1 by dividing for the fov size.

In [ ]:
# Normalize centroids between 0 and 1.
norm_factor = np.array([train_fov_size, train_fov_size])
df_train_dataset.loc[:, df_train_dataset.columns.str.contains("centroid")] = (
    df_train_dataset.loc[:, df_train_dataset.columns.str.contains("centroid")] / norm_factor
)

To train MAGIK with the simulated trajectories, you need to produce a graph representation with the function `GraphFromTrajectories`, defined in the utility file `utils.py`.

In [ ]:
# Instance the graph constructor.
graph_constructor = utils.GraphFromTrajectories(
    connectivity_radius=0.1,
    max_frame_distance=3,
)

# Generate graph from training data using graph constructor.
train_dataset_graph = graph_constructor(
    df=df_train_dataset,
)
print(train_dataset_graph)

Define the augmentation pipeline and the dataloader.

In [ ]:
# Initialize the graph dataset class.
# Specify augmentations with transform,
# NodeDropout() should be last.
train_dataset = utils.GraphDataset(
    train_dataset_graph,
    dataset_size=train_dataset_size,
    Dt=5,
    transform=Compose(
        [
            utils.RandomRotation(),
            utils.RandomFlip(),
            utils.NodeDropout(),
        ]
    )
)

# Initialize the training data loader.
train_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    drop_last=True,
)

Define the hyperparameters of the architecture.

In [ ]:
magik = dl.GraphToEdgeMAGIK(
    [96,] * 4,1,
    out_activation=torch.nn.Sigmoid
)

magik.encoder[0].configure(
    hidden_features=[32, 64],
    out_features=96,
    out_activation=torch.nn.ReLU,
)

magik.encoder[1].configure(
    hidden_features=[32, 64],
    out_features=96,
    out_activation=torch.nn.ReLU,
)

magik.head.configure(hidden_features=[64, 32]);

classifier_magik = dl.BinaryClassifier(
    model=magik,
    optimizer=dl.Adam(lr=1e-3),
).build()


# Set training parameters and train.
trainer_magik = dl.Trainer(max_epochs=200, accelerator="auto")

Train the model or load preexisting weights.

In [ ]:
# Define whether to force training even if weights exist.
force_training = True

# Define folder and model path.
folder_name = "MAGIK"
magik_model_path = os.path.join(folder_name, "magik_weights.pth")

# Train or load weights.
if not os.path.isfile(magik_model_path) or force_training:
    print("Training MAGIK model (either forced or weights not found).")

    # Ensure save directory exists.
    os.makedirs(folder_name, exist_ok=True)

    # Train the model.
    trainer_magik.fit(classifier_magik, train_loader)

    # Save trained weights.
    torch.save(magik.state_dict(), magik_model_path)
    print(f"Saved MAGIK weights to '{magik_model_path}'.")
else:
    # Load pre-trained weights.
    magik.load_state_dict(torch.load(magik_model_path, weights_only=True))
    print(f"Loaded preexisting MAGIK weights from '{magik_model_path}'.")

# Set the model to evaluation mode.
classifier_magik.eval();


### Linking Localizations in Simulations

You'll use the trajectories in the simulated video as the test dataset for the trained model of MAGIK. First, format the simulated dataframe. Then, convert the localizations corresponding to the simulated trajectories into a graph.

In [ ]:
# Rename for compatibility with label format.
df_sim_video_formatted = df_sim_video.rename(columns={"x": "centroid-0", "y": "centroid-1"})

# Add label, set, and solution columns.
df_sim_video_formatted[["label", "set", "solution"]] = 0

# Normalize coordinates to [0, 1].
frame_height, frame_width, _ = sim_image.shape
df_sim_video_formatted[["centroid-0", "centroid-1"]] /= [frame_width, frame_height]

# Generate a graph from graph_constructor. As test_graph returns a list of
#  graphs, we select the first element from the list as it only has 1 element.
sim_video_graph = graph_constructor(df=df_sim_video_formatted)[0]

Apply the trained model of MAGIK to predict the edge features.

In [ ]:
# Perform prediction on test graph.
sim_trajs_edges_pred_method3  = classifier_magik(sim_video_graph)
sim_trajs_edges_pred_method3  = sim_trajs_edges_pred_method3.detach().numpy() > 0.5

Get the ground-truth edge features and, as a first performance metrics, use the F1-score for the classification of the edges.

In [ ]:
# Get the ground truth edges from the graph.
sim_trajs_edges_gt = sim_video_graph.y

# Compute the F1 score.
F1 = f1_score(sim_trajs_edges_gt, sim_trajs_edges_pred_method3)
print(f"Test F1 score: {F1}")

The edge feature can be used to obtain the predicted trajectories using the dedicate class.

In [ ]:
# Compute the trajectories from the predicted edges.
trajectory_constructor = utils.ComputeTrajectories()
sim_trajs_pred_method3 = trajectory_constructor(
    sim_video_graph,
    sim_trajs_edges_pred_method3.squeeze(),
)

# Convert the predicted trajectories to a list format.
sim_trajs_pred_method3_list = utils.make_list(sim_trajs_pred_method3, sim_video_graph, sim_image_size)

Create a video with overlayed localizations and trajectories.

In [ ]:
# Create a video with the predicted trajectories.
sim_video_method3_results = utils.make_video_with_trajs(
    trajs_pred_list = sim_trajs_pred_method3_list,
    video = sim_video,
    fov_size = sim_image_size,
    trajs_gt_list = sim_trajs_gt_list,
)

sim_video_method3_results

### Evaluating Linking Performance

Evaluate the overall performance of the tracking.

In [ ]:
# Evaluate performance metrics.
utils.trajectory_metrics(
    sim_trajs_gt_list,
    sim_trajs_pred_method3_list,
    eps=5,
);

Display the reconstructed trajectories together with the groud truth.

In [ ]:
#  Compute the total squared distance between all trajectories to match
#  predicted trajectories with ground truth.
matched_pairs, _, _ = utils.trajectory_assignment(
    sim_trajs_gt_list,
    sim_trajs_pred_method3_list,
    eps=5,
)

# Plot the trajectories.
utils.plot_trajectory_matches(sim_trajs_gt_list, sim_trajs_pred_method3_list, matched_pairs)

Calculate the time-averaged MSD for all the trajectories and compare curves obtained for matching trajectories (same color).

In [ ]:
utils.plot_TAMSDs(
    trajs_pred = sim_trajs_pred_method3_list,
    trajs_gt = sim_trajs_gt_list,
    matched_pairs = matched_pairs,
) 

### Linking Localizations in Experiments

Apply the same steps to track the experiment and visualize the results.

In [ ]:
# Rename for compatibility with label format.
df_exp_video_formatted = df_exp_video.rename(columns={"x": "centroid-0", "y": "centroid-1"})

# Add label, set, and solution columns.
df_exp_video_formatted[["label", "set", "solution"]] = 0

# Normalize coordinates to [0, 1].
frame_height, frame_width, _ = sim_image.shape
df_exp_video_formatted[["centroid-0", "centroid-1"]] /= [frame_width, frame_height]

# Generate a graph from graph_constructor. As test_graph returns a list of
#  graphs, we select the first element from the list as it only has 1 element.
exp_video_graph = graph_constructor(df=df_exp_video_formatted)[0]

# Perform prediction on graph.
exp_trajs_edges_pred_method3  = classifier_magik(exp_video_graph)
exp_trajs_edges_pred_method3  = exp_trajs_edges_pred_method3.detach().numpy() > 0.5

# Compute the trajectories from the predicted edges.
trajectory_constructor = utils.ComputeTrajectories()
sim_trajs_pred_method3 = trajectory_constructor(
    exp_video_graph,
    exp_trajs_edges_pred_method3.squeeze(),
)

# Convert the predicted trajectories to a list format.
exp_trajs_pred_method3_list = utils.make_list(sim_trajs_pred_method3, exp_video_graph, exp_image_size)

# Create a video with the predicted trajectories.
exp_video_method3_results = utils.make_video_with_trajs(
    trajs_pred_list = exp_trajs_pred_method3_list,
    video = exp_video,
    fov_size = exp_image_size,
)

exp_video_method3_results

Calculate the time-averaged MSD for the trajectories.

In [ ]:
utils.plot_TAMSDs(
    trajs_pred = exp_trajs_pred_method3_list,
) 